In [1]:
import os
import json
import numpy as np
from skimage import io, color

In [2]:
def images_to_numpy_array(folder_path):
    image_arrays = []
    labels = []

    # Iterate through each folder (assuming each folder represents a label)
    for label in os.listdir(folder_path):
        label_path = os.path.join(folder_path, label)
        if not os.path.isdir(label_path):
            continue

        # Read images from each folder
        for filename in os.listdir(label_path):
            image_path = os.path.join(label_path, filename)
            if os.path.isfile(image_path) and filename.endswith(('.png')):
                # Open image using scikit-image
                image = io.imread(image_path, as_gray=True)

                # Convert image to numpy array
                image_array = np.array(image)

                # Append to list
                image_arrays.append(image_array)
                labels.append(label.replace('_', ''))
                # print(f'Image label: {label.replace("_", "")}, image_file: {image_path}')

    # Convert lists to numpy arrays
    image_arrays = np.array(image_arrays)
    labels = np.array(labels)

    return image_arrays, labels

In [3]:
images, labels = images_to_numpy_array('dataset')
print(images.shape)

(10360, 32, 32)


In [4]:
import os
import datetime
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataloader import default_collate
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torchvision import transforms

from myutils import *
from keras.preprocessing.image import ImageDataGenerator

/Users/ram/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
MODEL_SAVE_FOLDER = 'model'

In [6]:
class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [7]:
# Load images and labels
folder_path = "dataset"
images, eng_labels = images_to_numpy_array(folder_path)

# Normalize images and encode labels
images = images.astype("float32") / 255.0
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(eng_labels)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Create a dictionary of labels and their inverse transformed values
label_dict = [label_encoder.inverse_transform([label])[0] for label in set(labels)]

# Write the dictionary to a file
with open('labels.list', 'w') as file:
    json.dump(label_dict, file)

In [8]:
# Define transformations for data augmentation (if needed)
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((32,32)),
    transforms.ToTensor(),
])

# Create custom datasets
train_dataset = CustomDataset(X_train, y_train, transform=transform)
test_dataset = CustomDataset(X_test, y_test, transform=transform)

# Define dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [9]:
# Initialize the model, loss function, and optimizer
model = create_net()
print(f'created network for labelling {len(set(labels))} classes')
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

num_epochs = 600
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")


created network for labelling 70 classes
Epoch [1/600], Loss: 4.2563
Epoch [2/600], Loss: 4.2496
Epoch [3/600], Loss: 4.2492
Epoch [4/600], Loss: 4.2489
Epoch [5/600], Loss: 4.2488
Epoch [6/600], Loss: 4.2487
Epoch [7/600], Loss: 4.2486
Epoch [8/600], Loss: 4.2485
Epoch [9/600], Loss: 4.2485
Epoch [10/600], Loss: 4.2484
Epoch [11/600], Loss: 4.2484
Epoch [12/600], Loss: 4.2484
Epoch [13/600], Loss: 4.2484
Epoch [14/600], Loss: 4.2483
Epoch [15/600], Loss: 4.2484
Epoch [16/600], Loss: 4.2483
Epoch [17/600], Loss: 4.2483
Epoch [18/600], Loss: 4.2484
Epoch [19/600], Loss: 4.2484
Epoch [20/600], Loss: 4.2483
Epoch [21/600], Loss: 4.2483
Epoch [22/600], Loss: 4.2483


KeyboardInterrupt: 

In [ ]:
# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        print(outputs.shape)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"\nAccuracy on test set: {accuracy:.4f}")

torch.Size([32, 70])
torch.Size([32, 70])
torch.Size([32, 70])
torch.Size([32, 70])
torch.Size([32, 70])
torch.Size([32, 70])
torch.Size([18, 70])

Accuracy on test set: 0.6857


In [ ]:
import datetime
# Save the model to a file
# Save the model to a file with a timestamp-based name
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
if not os.path.exists(MODEL_SAVE_FOLDER):
    os.makedirs(MODEL_SAVE_FOLDER)
filename = MODEL_SAVE_FOLDER+'/'+f"model_{timestamp}.pth"
torch.save(model.state_dict(), filename)